# In this notebook i try to scrap the whole documentation of langchain and save it into txt files to later on use it for the chatbot

## the text files are stored in a chroma db vecor data base

In [3]:
# import libraries
from bs4 import BeautifulSoup
import requests
import re

In [4]:
# Function for getting the text data from a website url
def get_data(url):
	r = requests.get(url)
	return r.text

get_data('https://python.langchain.com/en/latest/index.html')

'<!doctype html>\n<html lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<meta name="generator" content="Docusaurus v2.4.0">\n<title data-rh="true">🦜️🔗 Langchain</title><meta data-rh="true" property="og:title" content="🦜️🔗 Langchain"><meta data-rh="true" name="viewport" content="width=device-width,initial-scale=1"><meta data-rh="true" name="twitter:card" content="summary_large_image"><meta data-rh="true" property="og:image" content="https://python.langchain.com/img/parrot-chainlink-icon.png"><meta data-rh="true" name="twitter:image" content="https://python.langchain.com/img/parrot-chainlink-icon.png"><meta data-rh="true" property="og:url" content="https://python.langchain.com/"><meta data-rh="true" name="docusaurus_locale" content="en"><meta data-rh="true" name="docusaurus_tag" content="default"><meta data-rh="true" name="docsearch:language" content="en"><meta data-rh="true" name="docsearch:docusaurus_tag" content="default"><link data-rh="true" rel="icon" href="/img/favicon.ico"><

In [5]:
# count links in a website
def get_links(website_link):
    html_data = get_data(website_link)
    soup = BeautifulSoup(html_data, "html.parser")
    list_links = []
    for link in soup.find_all("a", href=True):
        list_links.append(link["href"])
    return list_links

sub_links = get_links('https://python.langchain.com/en/latest/index.html')
sub_links

[]

In [6]:
len(sub_links)

0

In [7]:
# Add base path to all links
def add_base_path(website_link, list_links):
    list_links_with_base_path = []

    for link in list_links:

        if not link.startswith('/'):
            link_with_base_path = website_link + link
            list_links_with_base_path.append(link_with_base_path)

		# if link.startswith('https://') dont add base path
        elif link.startswith('http://'):
            list_links_with_base_path.append(link)

        elif link.startswith('.'):
            link_with_base_path = website_link + link.split('/')[-1]
            list_links_with_base_path.append(link_with_base_path)

    return list_links_with_base_path

link_list = add_base_path('https://python.langchain.com/en/latest/', sub_links)
link_list_print = print(link_list)

[]


In [207]:
def save_content(link_list):
    for i, link in enumerate(link_list):
        html_data = get_data(link)
        soup = BeautifulSoup(html_data, "html.parser")
        text = soup.get_text()

        # Remove the first 835 lines
        lines = text.splitlines()
        cleaned_text = "\n".join(lines[835:])

        # Get the first 3 words in the cleaned text
        words = cleaned_text.split()[:3]
        file_name_prefix = "_".join(words)

        # Replace special characters and spaces with an underscore
        file_name_prefix = re.sub(r"[^a-zA-Z0-9]+", "_", file_name_prefix)

        # Get the current working directory
        current_dir = os.getcwd()

        # Move up one level to the parent directory
        parent_dir = os.path.dirname(current_dir)

        # Set the path to the data folder
        data_folder = os.path.join(parent_dir, "data/langchain_doc")

        # Create the data folder if it doesn't exist
        if not os.path.exists(data_folder):
            os.makedirs(data_folder)

        # Set the path to the output file
        output_file = os.path.join(data_folder, f"{i}_{file_name_prefix}.txt")

        # Save the cleaned content to the output file
        with open(output_file, "w") as f:
            f.write(cleaned_text)

In [208]:
# save the content of the first 10 links into txt files with the name content_1.txt, content_2.txt, etc.
save_content(link_list)

# Q&A bot with langchain over a directory

In [8]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
import os
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

In [10]:
#create a new openai api key
os.environ["OPENAI_API_KEY"] = "sk-..."
# set up openai api key
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [11]:
# print number of txt files in directory
loader = DirectoryLoader('/Users/erictak/PycharmProjects/langchain/data/langchain_doc', glob="./*.txt")
doc = loader.load ( )
len(doc)

679

In [12]:
#splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(doc)

In [13]:
# count the number of chunks
len(texts)

5576

In [14]:
# print the first chunk
texts[0]

Document(page_content='Twitter\n\nContents\n\nInstallation and Setup\n\nDocument Loader\n\nTwitter#\n\nTwitter is an online social media and social networking service.\n\nInstallation and Setup#\n\npip install tweepy\n\nWe must initialize the loader with the Twitter API token, and we need to set up the Twitter username.\n\nDocument Loader#\n\nSee a usage example.\n\nfrom langchain.document_loaders import TwitterTweetLoader\n\nprevious\n\nTrello\n\nnext\n\nUnstructured\n\nContents\n\nInstallation and Setup\n\nDocument Loader\n\nBy Harrison Chase\n\n© Copyright 2023, Harrison Chase.\n\nLast updated on Jun 13, 2023.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/langchain_doc/592_Twitter_Contents_Installation.txt'})

# Data base creation with ChromaDB

https://www.youtube.com/watch?v=3yPBVii7Ct0

In [15]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [16]:
# persiste the db to disk
vectordb.persist()
vectordb = None

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [17]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


# create retriever

In [41]:
retriever = vectordb.as_retriever()

In [42]:
docs = retriever.get_relevant_documents("What to do when getting started?")

In [43]:
docs

[Document(page_content='Step 1: Create Tools# Agents are largely defined by the tools they can use. If you have a specific task you want the agent to accomplish, you have to give it access to the right tools. We have many tools natively in LangChain, so you should first look to see if any of them meet your needs. But we also make it easy to define a custom tool, so if you need custom tools you should absolutely do that.\n\n(Optional) Step 2: Modify Agent# The built-in LangChain agent types are designed to work well in generic situations, but you may be able to improve performance by modifying the agent implementation. There are several ways you could do this:\n\nModify the base prompt. This can be used to give the agent more context on how it should behave, etc. Modify the output parser. This is necessary if the agent is having trouble parsing the language model output.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/langchain_doc/644_Agents_Contents_Create.txt'}),

In [44]:
len(docs)

4

In [45]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [46]:
retriever.search_type

'similarity'

In [47]:
retriever.search_kwargs

{'k': 2}

# create a question answering chain

In [49]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [50]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [51]:
# full example
query = "What are the steps of the Quickstart Guide?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
 Step 1: Create Tools (Optional), Step 2: Modify Agent (Optional), Step 3: Modify Agent Executor.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/426_Agents_Contents_Create.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/644_Agents_Contents_Create.txt


In [53]:
# break it down
query = "What are all agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
#llm_response



> Entering new RetrievalQA chain...

> Finished chain.
 Custom Agent, Custom LLM Agent, Custom LLM Agent (with a ChatModel), Custom MRKL Agent, Custom MultiAction Agent, Custom Agent with Tool Retrieval, Conversation Agent (for Chat Models), Conversation Agent MRKL, MRKL Chat, ReAct, Self Ask With Search, Structured Tool Chat Agent.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/382_Agents_Agents_Note.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/382_Agents_Agents_Note.txt


In [54]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f8dba0c1ff0>)

In [55]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Deleteing the DB

In [250]:
!zip -r db.zip ./db

updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/index/ (stored 0%)
updating: db/index/index_metadata_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 5%)
updating: db/index/id_to_uuid_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 37%)
updating: db/index/uuid_to_id_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 39%)
updating: db/index/index_b9a5e02f-ebd0-4b13-8858-b30b211c4546.bin (deflated 17%)
updating: db/index/index_d80886e4-65e1-4231-8c73-99ff58d68061.bin (deflated 17%)
updating: db/index/uuid_to_id_b9a5e02f-ebd0-4b13-8858-b30b211c4546.pkl (deflated 41%)
updating: db/index/id_to_uuid_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 32%)
updating: db/index/index_metadata_d80886e4-65e1-4231-8c73-99ff58d68061.pkl (deflated 5%)
updating: db/chroma-collections.parquet (deflated 50%)
updating: db/.DS_Store (deflated 96%)


In [251]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

# Starting again loading the db

In [57]:
!unzip db.zip

Archive:  db.zip
replace db/chroma-embeddings.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

In [59]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

Using embedded DuckDB with persistence: data will be stored in: db


#### Usung turbo GPT API

In [60]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [61]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [62]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [63]:
# full example
query = "What are the agent types?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
There are two main types of agents mentioned in the context: Action Agents and Plan-and-Execute Agents. Action Agents decide the actions to take and execute those actions one at a time, while Plan-and-Execute Agents first decide a plan of actions to take, and then execute those actions one at a time.


Sources:
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/639_Agents_Contents_Action.txt
/Users/erictak/PycharmProjects/langchain/data/langchain_doc/344_Agents_Contents_Action.txt


In [64]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [65]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


# Visualizing the Vector db
https://github.com/mtybadger/chromaviz?ref=reactjsexample.com

https://github.com/avantrio/chroma-viewer


In [18]:
from langchain.vectorstores import Chroma

from chromaviz import visualize_collection
visualize_collection(vectordb._collection)

Using embedded DuckDB without persistence: data will be transient
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2023 18:43:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 18:43:54] "GET /assets/index-8146af32.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 18:43:54] "GET /assets/index-26d05a53.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 18:43:54] "GET /assets/glasses-5d966a6f.svg HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 18:43:55] "GET /assets/glasses-5d966a6f.svg HTTP/1.1" 200 -


           0         1         2         3         4         5         6     \
0      0.007118  0.021829  0.017987 -0.029570 -0.021800  0.013447 -0.010238   
1      0.011453  0.032211 -0.007604 -0.039562 -0.010802  0.021170 -0.045379   
2     -0.013182  0.035104  0.014451 -0.023866 -0.014564  0.024778 -0.004816   
3      0.000610  0.015848  0.004164 -0.016822 -0.004514  0.005991  0.012949   
4     -0.021725  0.014808  0.008128 -0.034246 -0.019668  0.020385 -0.014632   
...         ...       ...       ...       ...       ...       ...       ...   
36761  0.005724  0.015840  0.012498 -0.033025 -0.012662  0.011221 -0.009066   
36762 -0.021673  0.014796  0.008102 -0.034167 -0.019629  0.020360 -0.014620   
36763 -0.007286  0.006825  0.010365 -0.008643 -0.027148  0.026459 -0.041569   
36764  0.002776  0.005819  0.016296 -0.014760 -0.023690  0.030904 -0.038353   
36765  0.016768  0.021518 -0.001173 -0.043916 -0.012074  0.010573 -0.000993   

           7         8         9     ...      1526 

127.0.0.1 - - [19/Jun/2023 18:45:25] "GET /data HTTP/1.1" 200 -
